# Initial Exploration

(inspired by [this Kaggle notebook](https://www.kaggle.com/code/akbareza/simple-music-recommendation-system))


use this to adjust https://pypi.org/project/implicit/0.2.1/ to define a sophisticated similarity scores

https://surpriselib.com/

https://making.lyst.com/lightfm/docs/home.html#:~:text=LightFM%20is%20a%20Python%20implementation,the%20traditional%20matrix%20factorization%20algorithms.

https://pypi.org/project/spotlight/


The implicit library provides several built-in similarity metrics that you can use to compute item-item similarity on your dataset. Some of these similarity metrics include:

Cosine similarity: Measures the cosine of the angle between two vectors and is commonly used to compute similarity between items based on user interactions.

Jaccard similarity: Measures the size of the intersection divided by the size of the union of two sets and is often used for binary data (e.g., user-item interactions).

Pearson correlation coefficient: Measures the linear correlation between two variables and can be used to capture linear relationships between items based on user interactions.

Adjusted cosine similarity: Similar to cosine similarity but accounts for differences in item means, making it suitable for scenarios where items have varying average ratings or interactions.

These similarity metrics can be easily computed using the similar_items function provided by the implicit library. You can experiment with different similarity metrics to find the one that best suits your recommendation task and dataset characteristics. Additionally, you can also explore custom similarity metrics tailored to your specific use case, leveraging the flexibility of the implicit library.

## Import Library and Dataset

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [20]:
#Main Raw Dataframe
df = pd.read_csv("../data/dataset.csv")
df.drop(columns='Unnamed: 0',inplace=True)

In [30]:
#Dataframe for getting year feature of songs
dfYear = pd.read_csv("../data/data.csv")
dfYear = dfYear[['id','year']]
dfYear['track_id'] = dfYear['id']
dfYear.drop(columns='id',inplace=True)

In [31]:
#Merge 2 Dataframe
df = pd.merge(df,dfYear,on='track_id')
display(df.info(),df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9654 entries, 0 to 9653
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          9654 non-null   object 
 1   artists           9654 non-null   object 
 2   album_name        9654 non-null   object 
 3   track_name        9654 non-null   object 
 4   popularity        9654 non-null   int64  
 5   duration_ms       9654 non-null   int64  
 6   explicit          9654 non-null   bool   
 7   danceability      9654 non-null   float64
 8   energy            9654 non-null   float64
 9   key               9654 non-null   int64  
 10  loudness          9654 non-null   float64
 11  mode              9654 non-null   int64  
 12  speechiness       9654 non-null   float64
 13  acousticness      9654 non-null   float64
 14  instrumentalness  9654 non-null   float64
 15  liveness          9654 non-null   float64
 16  valence           9654 non-null   float64


None

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,year
0,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.905,0.000071,0.1320,0.1430,181.740,3,acoustic,2018
1,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.469,0.000000,0.0829,0.1670,119.949,4,acoustic,2017
2,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.1470,2,-8.822,1,0.0355,0.857,0.000003,0.0913,0.0765,141.284,3,acoustic,2014
3,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.1470,2,-8.822,1,0.0355,0.857,0.000003,0.0913,0.0765,141.284,3,piano,2014
4,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,80,242946,False,0.703,0.4440,11,-9.331,1,0.0417,0.559,0.000000,0.0973,0.7120,150.960,4,acoustic,2008


In [32]:
# Duplicate Check
df[df.duplicated('track_id')==True]

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,year
3,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.147,2,-8.822,1,0.0355,0.85700,0.000003,0.0913,0.0765,141.284,3,piano,2014
5,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,80,242946,False,0.703,0.444,11,-9.331,1,0.0417,0.55900,0.000000,0.0973,0.7120,150.960,4,rock,2008
8,7k9GuJYLp2AzqokyEdwEw2,Ross Copperman,Hunger,Hunger,56,205594,False,0.442,0.632,1,-6.770,1,0.0295,0.42600,0.004190,0.0735,0.1960,78.899,4,piano,2015
11,3ILmwMefYZoQh5Cf5jeuUQ,Motohiro Hata,Documentary,透明だった世界,61,232360,False,0.373,0.914,0,-4.185,1,0.0565,0.07600,0.000000,0.6690,0.5600,168.210,4,j-pop,2010
12,3ILmwMefYZoQh5Cf5jeuUQ,Motohiro Hata,Documentary,透明だった世界,61,232360,False,0.373,0.914,0,-4.185,1,0.0565,0.07600,0.000000,0.6690,0.5600,168.210,4,j-rock,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9227,1TIiWomS4i0Ikaf9EKdcLn,Tove Lo,BLUE LIPS (lady wood phase II),disco tits,64,223794,True,0.790,0.728,7,-6.523,1,0.0612,0.02500,0.001060,0.1720,0.3330,110.059,4,synth-pop,2017
9236,6MDijuuArPJv1vbp7K1x3f,Miike Snow,iii,Genghis Khan,62,212146,False,0.688,0.847,10,-3.238,0,0.0508,0.04000,0.000003,0.5980,0.4530,93.979,4,synth-pop,2016
9242,3VZQshi4COChhXaz7cLP02,Icona Pop;Charli XCX,I Love It (feat. Charli XCX),I Love It (feat. Charli XCX),63,155337,True,0.714,0.905,8,-2.663,1,0.0289,0.00855,0.000025,0.1460,0.8380,125.927,4,synth-pop,2012
9420,1CgbwsrNDlFrRuk2ebQ7zr,ATB,Movin' Melodies,9Pm (Till I Come),64,193880,False,0.672,0.941,9,-6.427,0,0.0494,0.02760,0.926000,0.1490,0.7540,130.218,4,trance,1999


In [33]:
df[df['track_id']=='6Vc5wAMmXdKIAM7WUoEb7N']

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,year
2,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.147,2,-8.822,1,0.0355,0.857,0.000003,0.0913,0.0765,141.284,3,acoustic,2014
3,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.147,2,-8.822,1,0.0355,0.857,0.000003,0.0913,0.0765,141.284,3,piano,2014


## Pre-Processing

### Genre Encoding

In [34]:
# Crosstab Genre and Song

xtab_song = pd.crosstab(
    df['track_id'], 
    df['track_genre']
)

xtab_song = xtab_song*2

display(xtab_song.head(),len(xtab_song))

track_genre,acoustic,afrobeat,alt-rock,alternative,ambient,anime,black-metal,bluegrass,blues,brazil,breakbeat,british,cantopop,chicago-house,children,chill,classical,club,comedy,country,dance,dancehall,death-metal,deep-house,detroit-techno,disco,disney,drum-and-bass,dub,dubstep,edm,electro,electronic,emo,folk,french,funk,garage,german,gospel,goth,grindcore,groove,grunge,guitar,happy,hard-rock,hardcore,hardstyle,heavy-metal,hip-hop,honky-tonk,house,idm,indian,indie,indie-pop,industrial,j-dance,j-idol,j-pop,j-rock,jazz,k-pop,kids,latin,latino,malay,mandopop,metal,metalcore,minimal-techno,mpb,new-age,opera,pagode,party,piano,pop,pop-film,power-pop,progressive-house,psych-rock,punk,punk-rock,r-n-b,reggae,reggaeton,rock,rock-n-roll,rockabilly,sad,salsa,samba,sertanejo,show-tunes,singer-songwriter,ska,sleep,songwriter,soul,spanish,swedish,synth-pop,tango,techno,trance,trip-hop,turkish,world-music
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
003vvx7Niy0yvhvHt4a68B,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
00KfIFi2TpAaQGPbRbFbKJ,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
00Y9r5SfbDdEjJhNg7laQr,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
00hTMcTeaaMtjBCV30yAm9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
00iwQgjIgoGO94BRBDyClo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


6788

In [35]:
# Concatenate the encoded genre columns with the original dataframe

dfDistinct = df.drop_duplicates('track_id')
dfDistinct = dfDistinct.sort_values('track_id')
dfDistinct = dfDistinct.reset_index(drop=True)

xtab_song.reset_index(inplace=True)
data_encoded = pd.concat([dfDistinct, xtab_song], axis=1)
display(data_encoded.head(),len(data_encoded))

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,year,track_id,acoustic,afrobeat,alt-rock,alternative,ambient,anime,black-metal,bluegrass,blues,brazil,breakbeat,british,cantopop,chicago-house,children,chill,classical,club,comedy,country,dance,dancehall,death-metal,deep-house,detroit-techno,disco,disney,drum-and-bass,dub,dubstep,edm,electro,electronic,emo,folk,french,funk,garage,german,gospel,goth,grindcore,groove,grunge,guitar,happy,hard-rock,hardcore,hardstyle,heavy-metal,hip-hop,honky-tonk,house,idm,indian,indie,indie-pop,industrial,j-dance,j-idol,j-pop,j-rock,jazz,k-pop,kids,latin,latino,malay,mandopop,metal,metalcore,minimal-techno,mpb,new-age,opera,pagode,party,piano,pop,pop-film,power-pop,progressive-house,psych-rock,punk,punk-rock,r-n-b,reggae,reggaeton,rock,rock-n-roll,rockabilly,sad,salsa,samba,sertanejo,show-tunes,singer-songwriter,ska,sleep,songwriter,soul,spanish,swedish,synth-pop,tango,techno,trance,trip-hop,turkish,world-music
0,003vvx7Niy0yvhvHt4a68B,The Killers,Hot Fuss,Mr. Brightside,86,222973,False,0.352,0.911,1,-5.230,1,0.0747,0.00121,0.000000,0.0995,0.236,148.033,4,alt-rock,2004,003vvx7Niy0yvhvHt4a68B,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00KfIFi2TpAaQGPbRbFbKJ,Linkin Park,LIVING THINGS,IN MY REMAINS,63,200693,False,0.553,0.907,9,-5.623,0,0.0484,0.00401,0.000000,0.2660,0.451,100.967,4,alternative,2012,00KfIFi2TpAaQGPbRbFbKJ,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,00Y9r5SfbDdEjJhNg7laQr,Harold Faltermeyer;Steve Stevens,Top Gun - Motion Picture Soundtrack (Special E...,"Top Gun Anthem - From ""Top Gun"" Original Sound...",62,252200,False,0.423,0.651,0,-8.741,1,0.0406,0.00799,0.618000,0.3100,0.140,107.011,4,german,1999,00Y9r5SfbDdEjJhNg7laQr,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,00hTMcTeaaMtjBCV30yAm9,"Bobby Cruz;Ricardo ""Richie"" Ray",Felices Pascuas,Aguinaldo Navideño,27,253506,False,0.638,0.667,7,-9.010,1,0.1150,0.77900,0.000867,0.0419,0.900,107.023,4,salsa,1971,00hTMcTeaaMtjBCV30yAm9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00iwQgjIgoGO94BRBDyClo,Kudasaibeats,Falling,The Girl I Haven't Met,56,195000,False,0.785,0.222,8,-21.624,1,0.0418,0.86000,0.840000,0.1060,0.327,139.885,4,chill,2017,00iwQgjIgoGO94BRBDyClo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


6788

## Scaling Numerical Features

In [36]:
numerical_features = ['explicit', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence','year']
scaler = MinMaxScaler()
data_encoded[numerical_features] = scaler.fit_transform(data_encoded[numerical_features])

## Similarity Calculation

In [37]:
# Select the relevant columns for computing item similarities
calculatied_features = numerical_features + list(xtab_song.drop(columns='track_id').columns)

cosine_sim = cosine_similarity(data_encoded[calculatied_features], data_encoded[calculatied_features])

## Output

### Output function

In [38]:
def get_recommendations(title, N=5):
    indices = pd.Series(data_encoded.index, index=data_encoded['track_name']).drop_duplicates()

    try:
        idx = indices[title]
        try:
            len(idx)
            temp = 2
        except:
            temp = 1
    except KeyError:
        return "Song not found in the dataset."
    
    if temp == 2:
        idx = indices[title][0]
    else:
        idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:N+1]
    song_indices = [i[0] for i in sim_scores]
    recommended_songs = data_encoded[['track_name', 'artists', 'album_name']].iloc[song_indices]

    sim_scores_list = [i[1] for i in sim_scores]
    recommended_list = recommended_songs.to_dict(orient='records')
    for i, song in enumerate(recommended_list):
        song['similarity_score'] = sim_scores_list[i]
    
    return recommended_list


## Input and Recommendation Output

In [39]:
# Get the recommendations
recommended_songs = get_recommendations("Time", N=5)
if isinstance(recommended_songs, str):
    print(recommended_songs)
else:
    print("Recommended Songs:")
    for song in recommended_songs:
        print(f"Title: {song['track_name']}")
        print(f"Artist: {song['artists']}")
        print(f"Album: {song['album_name']}")
        print(f"Similarity Score: {song['similarity_score']:.2f}")
        print()

Recommended Songs:
Title: Suite: Judy Blue Eyes - 2005 Remaster
Artist: Crosby, Stills & Nash
Album: Crosby, Stills & Nash
Similarity Score: 1.00

Title: Red House
Artist: Jimi Hendrix
Album: Are You Experienced
Similarity Score: 1.00

Title: Heroes And Villains - Remastered 2001
Artist: The Beach Boys
Album: Smiley Smile (Remastered)
Similarity Score: 1.00

Title: The Wind Cries Mary
Artist: Jimi Hendrix
Album: Are You Experienced
Similarity Score: 0.99

Title: The Trial
Artist: Pink Floyd
Album: The Wall
Similarity Score: 0.99

